In [ ]:
!pip install yfinance ta plotly

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=bca08751118c2108180b717f25baeda5968003f14bb05be66c76cc13f128ed0b
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
import yfinance as yf
import pandas as pd
import time
from ta.trend import SMAIndicator, MACD
from ta.momentum import RSIIndicator
from IPython.display import clear_output, display
from zoneinfo import ZoneInfo
from datetime import datetime

# === Config ===
symbol = 'BSE.NS'
interval = '5m'
period = '5d'
refresh_seconds = 60

# === Init Signal Storage ===
signals = pd.DataFrame(columns=['Time (IST)', 'Signal', 'Price'])

while True:
    try:
        # === Fetch Data ===
        df = yf.download(tickers=symbol, interval=interval, period=period, progress=False)
        if df.empty:
            print("No data found. Retrying...")
            time.sleep(refresh_seconds)
            continue

        df.index = df.index.tz_convert('Asia/Kolkata')
        close_series = df['Close'].squeeze()

        # === Indicators ===
        df['SMA'] = SMAIndicator(close=close_series, window=20).sma_indicator()
        df['RSI'] = RSIIndicator(close=close_series, window=14).rsi()
        macd = MACD(close=close_series)
        df['MACD'] = macd.macd()
        df['MACD_signal'] = macd.macd_signal()

        # === Latest Row Values ===
        last_row = df.iloc[-1]
        timestamp = df.index[-1]
        close = last_row['Close'].item()
        sma = last_row['SMA'].item()
        rsi = last_row['RSI'].item()
        macd_val = last_row['MACD'].item()
        macd_signal = last_row['MACD_signal'].item()

        # === Signal Conditions ===
        signal = None
        if close > sma and rsi > 50 and macd_val > macd_signal:
            signal = 'BUY'
        elif close < sma and rsi < 50 and macd_val < macd_signal:
            signal = 'SELL'

        # === Save Signal if present ===
        if signal:
            signals = pd.concat([signals, pd.DataFrame([{
                'Time (IST)': timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                'Signal': signal,
                'Price': round(close, 2)
            }])], ignore_index=True)

        # === Display Table ===
        clear_output(wait=True)
        print(f"Last Updated: {datetime.now(ZoneInfo('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')} IST")
        display(signals.tail(20))

        time.sleep(refresh_seconds)

    except KeyboardInterrupt:
        print("Stopped manually.")
        break
    except Exception as e:
        print(f"Error: {e}")
        time.sleep(refresh_seconds)


Last Updated: 2025-04-15 15:25:33 IST


,Time (IST),Signal,Price
27,2025-04-15 15:05:00,BUY,5936.5
28,2025-04-15 15:05:00,BUY,5936.0
29,2025-04-15 15:05:00,BUY,5943.5
30,2025-04-15 15:05:00,BUY,5943.0
31,2025-04-15 15:10:00,BUY,5941.5
32,2025-04-15 15:10:00,BUY,5948.5
33,2025-04-15 15:10:00,BUY,5959.0
34,2025-04-15 15:10:00,BUY,5957.5
35,2025-04-15 15:10:00,BUY,5959.0
36,2025-04-15 15:15:00,BUY,5959.5
